# AgroPest-12 YOLOv8 baseline

Yolov8 NANO used

## Setup

In [1]:
# Imports
from ultralytics import YOLO
import torch
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

In [2]:
# Device (GPU if available)
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Device: {torch.cuda.get_device_name(0) if device == 0 else 'CPU'}")

Device: NVIDIA GeForce RTX 3060


In [ ]:
# Dataset config (adjust path if you move this notebook)
data_yaml_path = Path('../dataset/data.yaml')
with open(data_yaml_path, 'r') as f:
    data = yaml.safe_load(f)
names = data.get('names', [])
print('Classes:', names)

Classes: ['Ants', 'Bees', 'Beetles', 'Caterpillars', 'Earthworms', 'Earwigs', 'Grasshoppers', 'Moths', 'Slugs', 'Snails', 'Wasps', 'Weevils']


## Model Training

In [4]:
# Train (YOLOv8n default settings)
model = YOLO('yolov8n.pt')
results = model.train(
    data=str(data_yaml_path),
    epochs=50,
    imgsz=640,
    device=device,
    project='runs/detect',
    name='agropest_yolov8n_baseline',
    seed=42
)
print('Training done.')

Ultralytics 8.3.225  Python-3.12.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=..\dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=agropest_yolov8n_baseline3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=

## Validation

In [12]:
# Evaluate (validation) and show simple metrics and plots
val = model.val(data=str(data_yaml_path), split='val', device=device, imgsz=640, plots=True)
P = val.box.mp
R = val.box.mr
F1 = 2*P*R/(P+R)
cm = val.confusion_matrix.matrix  
A = cm.trace() / cm.sum()  

print('VALIDATION RESULTS:')
print(' Accuracy :', A)
print(' Precision:', P)
print(' Recall   :', R)
print(' F1       :', F1)
print(' mAP@0.5  :', val.box.map50)
print(' mAP@0.5:0.95:', val.box.map)

Ultralytics 8.3.225  Python-3.12.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 246.0108.1 MB/s, size: 48.5 KB)
val: Scanning C:\Users\jerry\OneDrive\Desktop\9517_Group_Project\dataset\valid\labels.cache... 1095 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1095/1095  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 69/69 9.0it/s 7.7s0.1s
                   all       1095       1341      0.824      0.712      0.771      0.451
                  Ants         96        178      0.808      0.638      0.703      0.283
                  Bees         99        110      0.897        0.8      0.835       0.37
               Beetles         89        100      0.658       0.62      0.634      0.287
          Caterpillars         77        139      0.838      0.453      0.513      0.267
            Earthworms         53         72      0.633      0.431      0.474   

## Testing

In [13]:
# Evaluate on test split (if present in data.yaml)
test = model.val(data=str(data_yaml_path), split='test', device=device, imgsz=640, plots=True)
P_t = test.box.mp
R_t = test.box.mr
F1_t = 2*P_t*R_t/(P_t+R_t)
cm_t = test.confusion_matrix.matrix  
A_t = cm_t.trace() / cm_t.sum()   

print('TEST RESULTS:')
print(' Accuracy :', A_t)
print(' Precision:', P_t)
print(' Recall   :', R_t)
print(' F1       :', F1_t)
print(' mAP@0.5  :', test.box.map50)
print(' mAP@0.5:0.95:', test.box.map)

Ultralytics 8.3.225  Python-3.12.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 410.9163.1 MB/s, size: 45.0 KB)
val: Scanning C:\Users\jerry\OneDrive\Desktop\9517_Group_Project\dataset\test\labels.cache... 546 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 546/546  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 8.6it/s 4.1s0.1s
                   all        546        689      0.788      0.743      0.768      0.455
                  Ants         54         87      0.843      0.713      0.736      0.332
                  Bees         40         44      0.847      0.879      0.925      0.476
               Beetles         41         44      0.714      0.736      0.681      0.371
          Caterpillars         46         93      0.656      0.441      0.545      0.252
            Earthworms         27         40      0.488       0.45      0.482      0